In [129]:
import numpy as np
import pandas as pd

In [130]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [131]:
# d_train.append(d_test, sort = False).info()
# d_train.info()
# nan in age, cabin, embarked (2)
# d_test.info()
# nan in age, fare, cabin

In [132]:
dataset =  pd.concat(objs=[train, test], axis=0, sort=False).reset_index(drop=True)

In [133]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
PassengerId    1309 non-null int64
Survived       891 non-null float64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


In [134]:
dataset.groupby('Sex')['Embarked'].value_counts(normalize=True)

Sex     Embarked
female  S           0.627155
        C           0.243534
        Q           0.129310
male    S           0.739027
        C           0.186240
        Q           0.074733
Name: Embarked, dtype: float64

In [135]:
# Show nan in Embarked column
# nans = lambda df: df[df['Embarked'].isnull()]
# nans(d_train)

In [136]:
# d_train = d_train.dropna(subset=['Embarked'])
dataset["Embarked"] = dataset["Embarked"].fillna('S')
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
PassengerId    1309 non-null int64
Survived       891 non-null float64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB
None


In [137]:
# mean of Fare under 20k in test set
mu = dataset[dataset.Fare < 20]['Fare'].mean()
# print(mu)

In [138]:
# Show nan in Embarked column
nans = lambda df: df[df['Fare'].isnull()]
nans(dataset)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1043,1044,NaN,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S


In [139]:
dataset.loc[dataset.Fare.isnull(), "Fare"] = mu
print(dataset.loc[dataset['PassengerId'] == (152+892)])

      PassengerId  Survived  Pclass                Name   Sex   Age  SibSp  \
1043         1044       NaN       3  Storey, Mr. Thomas  male  60.5      0   

      Parch Ticket      Fare Cabin Embarked  
1043      0   3701  9.727188   NaN        S  


In [140]:
def handleNaNAge(dataset):
    index_NaN_age = list(dataset["Age"][dataset["Age"].isnull()].index)

    for i in index_NaN_age :
        age_med = dataset["Age"].median()
        age_pred = dataset["Age"][((dataset['SibSp'] == dataset.iloc[i]["SibSp"]) & (dataset['Parch'] == dataset.iloc[i]["Parch"]) & (dataset['Pclass'] == dataset.iloc[i]["Pclass"]))].median()
        if not np.isnan(age_pred) :
            dataset['Age'].iloc[i] = age_pred
        else :
            dataset['Age'].iloc[i] = age_med
        
        
handleNaNAge(dataset)

/home/krigare/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [141]:
def sF(df):
    for i in range(1,5):
        df.loc[df['Fare'].between((i-1)*20, 20*i, inclusive=True), 'FareBin'] = i-1
    df.loc[df['Fare'].between(80, df['Fare'].max(), inclusive=True), 'FareBin'] = 4
    df["FareBin"] = df["FareBin"].astype(int)
    return df

def sA(df):
    for i in range(1,6):
        df.loc[df['Age'].between((i-1)*20, 20*i, inclusive=True), 'AgeBin'] = i-1
    df.loc[df['Age'].between(80, df['Age'].max(), inclusive=True), 'FareBin'] = 5
    df["AgeBin"] = df["AgeBin"].astype(int)
    return df
#sF(dataset)
#sA(dataset)
#print()

In [143]:
dataset.loc[dataset.Parch > 0, "Parch2"] = 1
dataset.loc[dataset.Parch == 0, "Parch2"] = 0

# print(d_test.loc[d_test['PassengerId'] == (152+892)])
#dataset.groupby('AgeBin')['Parch2'].value_counts(normalize=True)


In [145]:
dataset.loc[dataset.SibSp > 0, "SibSp2"] = 1
dataset.loc[dataset.SibSp == 0, "SibSp2"] = 0

# print(d_test.loc[d_test['PassengerId'] == (152+892)])
#dataset.groupby('AgeBin')['SibSp2'].value_counts(normalize=True)


In [146]:
dataset.loc[dataset.Age < 15, "IsChild"] = 1
dataset.loc[dataset.Age >= 15, "IsChild"] = 0
print(dataset.sample(10))


      PassengerId  Survived  Pclass  \
1035         1036       NaN       1   
356           357       1.0       1   
704           705       0.0       3   
974           975       NaN       3   
1094         1095       NaN       2   
204           205       1.0       3   
571           572       1.0       1   
242           243       0.0       2   
804           805       1.0       3   
85             86       1.0       3   

                                                   Name     Sex   Age  SibSp  \
1035  Lindeberg-Lind, Mr. Erik Gustaf (Mr Edward Lin...    male  42.0      0   
356                         Bowerman, Miss. Elsie Edith  female  22.0      0   
704                             Hansen, Mr. Henrik Juul    male  26.0      1   
974                                Demetri, Mr. Marinko    male  25.0      0   
1094                         Quick, Miss. Winifred Vera  female   8.0      1   
204                            Cohen, Mr. Gurshon "Gus"    male  18.0      0   
571       

In [147]:
print(dataset.sample(10))



      PassengerId  Survived  Pclass  \
731           732       0.0       3   
234           235       0.0       2   
1247         1248       NaN       1   
189           190       0.0       3   
359           360       1.0       3   
1024         1025       NaN       3   
99            100       0.0       2   
1207         1208       NaN       1   
488           489       0.0       3   
982           983       NaN       3   

                                                Name     Sex   Age  SibSp  \
731                         Hassan, Mr. Houssein G N    male  11.0      0   
234                Leyson, Mr. Robert William Norman    male  24.0      0   
1247  Brown, Mrs. John Murray (Caroline Lane Lamson)  female  59.0      2   
189                              Turcin, Mr. Stjepan    male  36.0      0   
359                Mockler, Miss. Helen Mary "Ellie"  female  25.0      0   
1024                           Thomas, Mr. Charles P    male  25.0      1   
99                             

In [148]:
# Squeeze SibSp and Parch into one
dataset['SP'] = dataset['SibSp'] + dataset['Parch']
#d_test['SP'] = d_test['SibSp'] + d_test['Parch']
dataset = dataset.drop(columns=['SibSp','Parch', 'Parch2', 'SibSp2'], axis=1)
#d_test = d_test.drop(columns=['SibSp','Parch'], axis=1)

dataset.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Cabin,Embarked,IsChild,SP
1047,1048,NaN,1,"Bird, Miss. Ellen",female,29.00,PC 17483,221.7792,C97,S,0.0,0
593,594,0.0,3,"Bourke, Miss. Mary",female,26.00,364848,7.7500,NaN,Q,0.0,2
1125,1126,NaN,1,"Cumings, Mr. John Bradley",male,39.00,PC 17599,71.2833,C85,C,0.0,1
597,598,0.0,3,"Johnson, Mr. Alfred",male,49.00,LINE,0.0000,NaN,S,0.0,0
836,837,0.0,3,"Pasic, Mr. Jakob",male,21.00,315097,8.6625,NaN,S,0.0,0
425,426,0.0,3,"Wiseman, Mr. Phillippe",male,25.00,A/4. 34244,7.2500,NaN,S,0.0,0
212,213,0.0,3,"Perkin, Mr. John Henry",male,22.00,A/5 21174,7.2500,NaN,S,0.0,0
133,134,1.0,2,"Weisz, Mrs. Leopold (Mathilde Francoise Pede)",female,29.00,228414,26.0000,NaN,S,0.0,1
803,804,1.0,3,"Thomas, Master. Assad Alexander",male,0.42,2625,8.5167,NaN,C,1.0,1
438,439,0.0,1,"Fortune, Mr. Mark",male,64.00,19950,263.0000,C23 C25 C27,S,0.0,5


In [149]:
# encode Sex
dataset['Sex'].replace(['female','male'], [0,1], inplace = True)

 


In [150]:
# encode Embarked
dataset['Embarked'].replace(['Q','S', 'C'], [0,1,2], inplace = True)
dataset['Embarked'].unique()

array([1, 2, 0])

In [151]:
# Get Title from Name
dataset_title = [i.split(",")[1].split(".")[0].strip() for i in dataset["Name"]]
dataset["Title"] = pd.Series(dataset_title)
dataset["Title"].value_counts()

Mr              757
Miss            260
Mrs             197
Master           61
Rev               8
Dr                8
Col               4
Major             2
Mlle              2
Ms                2
Dona              1
Mme               1
Sir               1
the Countess      1
Lady              1
Jonkheer          1
Capt              1
Don               1
Name: Title, dtype: int64

In [152]:
# Convert to categorical values Title 
dataset["Title"] = dataset["Title"].replace(['Lady', 'Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer'], 'Rare')
dataset["Title"] = dataset["Title"].map({"Master":0, "Dona":1 ,"the Countess":1, "Countess":1, "Miss":1, "Ms" : 1 , "Mme":1, "Mlle":1, "Mrs":1, "Mr":2, "Rare":3})
dataset["Title"] = dataset["Title"].astype(int)

In [153]:
# Drop Name variable
dataset.drop(labels = ["Name"], axis = 1, inplace = True)

In [154]:
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
PassengerId    1309 non-null int64
Survived       891 non-null float64
Pclass         1309 non-null int64
Sex            1309 non-null int64
Age            1309 non-null float64
Ticket         1309 non-null object
Fare           1309 non-null float64
Cabin          295 non-null object
Embarked       1309 non-null int64
IsChild        1309 non-null float64
SP             1309 non-null int64
Title          1309 non-null int64
dtypes: float64(4), int64(6), object(2)
memory usage: 122.8+ KB
None


In [155]:
# Create new feature of family size
dataset['Single'] = dataset['SP'].map(lambda s: 1 if s == 1 else 0)
dataset['SmallF'] = dataset['SP'].map(lambda s: 1 if  s == 2  else 0)
dataset['MedF'] = dataset['SP'].map(lambda s: 1 if 3 <= s <= 4 else 0)
dataset['LargeF'] = dataset['SP'].map(lambda s: 1 if s >= 5 else 0)

In [156]:
dataset.head()

,PassengerId,Survived,Pclass,Sex,Age,Ticket,Fare,Cabin,Embarked,IsChild,SP,Title,Single,SmallF,MedF,LargeF
0,1,0.0,3,1,22.0,A/5 21171,7.2500,NaN,1,0.0,1,2,1,0,0,0
1,2,1.0,1,0,38.0,PC 17599,71.2833,C85,2,0.0,1,1,1,0,0,0
2,3,1.0,3,0,26.0,STON/O2. 3101282,7.9250,NaN,1,0.0,0,1,0,0,0,0
3,4,1.0,1,0,35.0,113803,53.1000,C123,1,0.0,1,1,1,0,0,0
4,5,0.0,3,1,35.0,373450,8.0500,NaN,1,0.0,0,2,0,0,0,0


In [157]:
# convert to indicator values Title and Embarked 
dataset = pd.get_dummies(dataset, columns = ["Title"])
dataset = pd.get_dummies(dataset, columns = ["Embarked"], prefix="Em")

In [158]:
# Replace the Cabin number by the type of cabin 'X' if not
dataset["Cabin"] = pd.Series([i[0] if not pd.isnull(i) else 'X' for i in dataset['Cabin'] ])
dataset = pd.get_dummies(dataset, columns = ["Cabin"],prefix="Cabin")


In [159]:
## Treat Ticket by extracting the ticket prefix. When there is no prefix it returns X. 
def exTiPref(dataset):
    Ticket = []
    for i in list(dataset.Ticket):
        if not i.isdigit() :
            Ticket.append(i.replace(".","").replace("/","").strip().split(' ')[0]) #Take prefix
        else:
            Ticket.append("X")        
    dataset["Ticket"] = Ticket
exTiPref(dataset)
dataset["Ticket"].head()

0        A5
1        PC
2    STONO2
3         X
4         X
Name: Ticket, dtype: object

In [160]:
dataset = pd.get_dummies(dataset, columns = ["Ticket"], prefix="T")

In [161]:
# Create categorical values for Pclass
dataset["Pclass"] = dataset["Pclass"].astype("category")
dataset = pd.get_dummies(dataset, columns = ["Pclass"],prefix="Pc")

In [162]:
# Create categorical values for FareBin, AgeBin
#dataset["FareBin"] = dataset["FareBin"].astype("category")
#dataset = pd.get_dummies(dataset, columns = ["FareBin"],prefix="Fb")

#dataset["AgeBin"] = dataset["AgeBin"].astype("category")
#dataset = pd.get_dummies(dataset, columns = ["AgeBin"],prefix="Ab")

In [163]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 67 columns):
PassengerId    1309 non-null int64
Survived       891 non-null float64
Sex            1309 non-null int64
Age            1309 non-null float64
Fare           1309 non-null float64
IsChild        1309 non-null float64
SP             1309 non-null int64
Single         1309 non-null int64
SmallF         1309 non-null int64
MedF           1309 non-null int64
LargeF         1309 non-null int64
Title_0        1309 non-null uint8
Title_1        1309 non-null uint8
Title_2        1309 non-null uint8
Title_3        1309 non-null uint8
Em_0           1309 non-null uint8
Em_1           1309 non-null uint8
Em_2           1309 non-null uint8
Cabin_A        1309 non-null uint8
Cabin_B        1309 non-null uint8
Cabin_C        1309 non-null uint8
Cabin_D        1309 non-null uint8
Cabin_E        1309 non-null uint8
Cabin_F        1309 non-null uint8
Cabin_G        1309 non-null uint8
Cabin_T    

In [164]:
# Drop useless variables 
dataset.drop(labels = ["PassengerId"], axis = 1, inplace = True)

In [201]:
dataset["IsChild"] = dataset["IsChild"].astype(int)


In [203]:
train = dataset[:len(train)]
test = dataset[len(train):]
test.drop(labels=["Survived"],axis = 1,inplace=True)

/home/krigare/anaconda3/lib/python3.5/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [204]:
test.head()

,Sex,Age,Fare,IsChild,SP,Single,SmallF,MedF,LargeF,Title_0,...,T_STONO,T_STONO2,T_STONOQ,T_SWPP,T_WC,T_WEP,T_X,Pc_1,Pc_2,Pc_3
891,1,34.5,7.8292,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
892,0,47.0,7.0000,0,1,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
893,1,62.0,9.6875,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
894,1,27.0,8.6625,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
895,0,22.0,12.2875,0,2,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1


In [205]:
# Feature scaling -> Fare
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train[['Fare', 'Age']] = sc.fit_transform(train[['Fare', 'Age']])
#print(d_train.sample(3))
test[['Fare', 'Age']] = sc.transform(test[['Fare', 'Age']])
#print(d_test.head(3))
#print(sc.transform(d_test[['Fare']]))
#sc.transform(d_train['Fare'])

/home/krigare/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/krigare/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/krigare/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.p

In [206]:
## Separate train features and label 

train["Survived"] = train["Survived"].astype(int)

Y = train["Survived"]

X = train.drop(labels = ["Survived"],axis = 1)

/home/krigare/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [207]:
# Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2137)

In [236]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(712, 65) (179, 65) (712,) (179,)
<class 'pandas.core.frame.DataFrame'>


In [209]:
from sklearn.svm import SVC
svm = SVC(C=500, kernel ='rbf', random_state = 7, probability = True)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)


In [210]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
prec = (cm[0,0] + cm[1,1]) / sum(sum(cm))
print(prec)

0.8268156424581006


In [212]:
# Logistic regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state = 0)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

In [213]:
from sklearn.metrics import confusion_matrix
cmlr = confusion_matrix(y_test, y_pred)
preclr = (cmlr[0,0] + cmlr[1,1]) / sum(sum(cmlr))
print(preclr)

0.8212290502793296


In [214]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=50, max_depth=6, random_state=2137)
clf.fit(X_train, y_train)
y_pred_rf = clf.predict(X_test)

In [248]:
from xgboost import XGBClassifier
xgb = XGBClassifier(max_depth=6, reg_lambda=1000, subsample=0.9, random_state = 2100)
xgb.fit(X_train.values, y_train.values)

# Predicting the Test set results
y_pred_XGB = xgb.predict(X_test.values)
print(y_pred_XGB[0:3])

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm_XGB = confusion_matrix(y_test.values, y_pred_XGB)
print(cm_XGB)
# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = xgb, X = X_train.values, y = y_train.values, cv = 10)
print(accuracies.mean())
print(accuracies.std())

/home/krigare/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/krigare/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[0 0 1]
[[94 13]
 [20 52]]


/home/krigare/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/krigare/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/krigare/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/krigare/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.8314749608763693
0.04122870123298839


/home/krigare/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/krigare/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [216]:
print(clf.feature_importances_)

[1.84678575e-01 4.78940891e-02 7.75027707e-02 1.02303049e-02
 3.78701760e-02 5.69023659e-03 7.72103935e-03 3.99823113e-03
 2.04702870e-02 7.85065946e-03 1.44514858e-01 1.97620147e-01
 4.64767665e-03 5.09315867e-03 1.01030484e-02 1.01325896e-02
 1.50281159e-03 8.36999709e-03 6.04537060e-03 6.98687187e-03
 7.28454322e-03 1.27848108e-03 4.76860526e-04 1.74225386e-04
 2.59790732e-02 0.00000000e+00 6.63678835e-04 1.97233599e-03
 0.00000000e+00 0.00000000e+00 0.00000000e+00 9.75455451e-04
 4.44613541e-03 2.55599418e-06 2.47489225e-04 0.00000000e+00
 0.00000000e+00 7.65090718e-04 0.00000000e+00 6.55801703e-03
 4.18721014e-04 4.31991856e-04 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.87792843e-06 1.04236348e-03
 6.25586925e-05 1.29275374e-03 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.49426629e-03 6.61651689e-05 2.46516376e-03
 2.79963977e-04 0.00000000e+00 4.49898681e-03 2.08394376e-03
 1.70914917e-03 5.58110477e-03 3.58676876e-02 1.90648067e-02
 7.38916549e-02]


In [217]:
cmrf = confusion_matrix(y_test, y_pred_rf)
precrf = (cmrf[0,0] + cmrf[1,1]) / sum(sum(cmrf))
print(precrf)

0.8212290502793296


In [179]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam


model = Sequential()
model.add(Dense(12, input_dim=X.shape[1], activation='relu', kernel_initializer='uniform'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu', kernel_initializer='uniform'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu', kernel_initializer='uniform'))
#model.add(Dropout(0.2))
model.add(Dense(12, activation='relu', kernel_initializer='uniform'))
model.add(Dense(8, activation='relu', kernel_initializer='uniform'))
model.add(Dropout(0.1))
model.add(Dense(4, activation='relu', kernel_initializer='uniform'))
#model.add(Dropout(0.2))
model.add(Dense(4, activation='relu', kernel_initializer='uniform'))
#model.add(Dropout(0.2))
model.add(Dense(2, activation='relu', kernel_initializer='uniform'))
#model.add(Dropout(0.2))
# model.add(Dropout(0.1))
model.add(Dense(2, activation='relu', kernel_initializer='uniform'))
model.add(Dense(1, activation='sigmoid'))

adam = Adam(lr = 0.001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=32, epochs=500, validation_split = 0.2)

Train on 569 samples, validate on 143 samples
Epoch 1/500
569/569 [==============================] - 2s 4ms/step - loss: 0.6920 - acc: 0.6046 - val_loss: 0.6887 - val_acc: 0.6853
Epoch 2/500
569/569 [==============================] - 0s 156us/step - loss: 0.6895 - acc: 0.6046 - val_loss: 0.6837 - val_acc: 0.6853
Epoch 3/500
569/569 [==============================] - 0s 140us/step - loss: 0.6867 - acc: 0.6046 - val_loss: 0.6793 - val_acc: 0.6853
Epoch 4/500
569/569 [==============================] - 0s 186us/step - loss: 0.6845 - acc: 0.6046 - val_loss: 0.6741 - val_acc: 0.6853
Epoch 5/500
569/569 [==============================] - 0s 185us/step - loss: 0.6820 - acc: 0.6046 - val_loss: 0.6699 - val_acc: 0.6853
Epoch 6/500
569/569 [==============================] - 0s 228us/step - loss: 0.6800 - acc: 0.6046 - val_loss: 0.6651 - val_acc: 0.6853
Epoch 7/500
569/569 [==============================] - 0s 163us/step - loss: 0.6780 - acc: 0.6046 - val_loss: 0.6605 - val_acc: 0.6853
Epoch 8/500

Epoch 61/500
569/569 [==============================] - 0s 143us/step - loss: 0.4181 - acc: 0.8453 - val_loss: 0.4983 - val_acc: 0.7832
Epoch 62/500
569/569 [==============================] - 0s 182us/step - loss: 0.4212 - acc: 0.8453 - val_loss: 0.4984 - val_acc: 0.7762
Epoch 63/500
569/569 [==============================] - 0s 169us/step - loss: 0.4330 - acc: 0.8612 - val_loss: 0.4897 - val_acc: 0.7832
Epoch 64/500
569/569 [==============================] - 0s 184us/step - loss: 0.4207 - acc: 0.8541 - val_loss: 0.4935 - val_acc: 0.7972
Epoch 65/500
569/569 [==============================] - 0s 221us/step - loss: 0.4116 - acc: 0.8541 - val_loss: 0.5026 - val_acc: 0.7902
Epoch 66/500
569/569 [==============================] - 0s 178us/step - loss: 0.4222 - acc: 0.8594 - val_loss: 0.4945 - val_acc: 0.7832
Epoch 67/500
569/569 [==============================] - 0s 139us/step - loss: 0.4085 - acc: 0.8576 - val_loss: 0.4966 - val_acc: 0.7832
Epoch 68/500
569/569 [==========================

569/569 [==============================] - 0s 197us/step - loss: 0.3614 - acc: 0.8752 - val_loss: 0.5528 - val_acc: 0.7762
Epoch 122/500
569/569 [==============================] - 0s 169us/step - loss: 0.3636 - acc: 0.8594 - val_loss: 0.5683 - val_acc: 0.7762
Epoch 123/500
569/569 [==============================] - 0s 158us/step - loss: 0.3693 - acc: 0.8664 - val_loss: 0.5588 - val_acc: 0.7762
Epoch 124/500
569/569 [==============================] - 0s 187us/step - loss: 0.3478 - acc: 0.8770 - val_loss: 0.5660 - val_acc: 0.7762
Epoch 125/500
569/569 [==============================] - 0s 190us/step - loss: 0.3668 - acc: 0.8717 - val_loss: 0.5632 - val_acc: 0.7832
Epoch 126/500
569/569 [==============================] - 0s 142us/step - loss: 0.3381 - acc: 0.8875 - val_loss: 0.5622 - val_acc: 0.7832
Epoch 127/500
569/569 [==============================] - 0s 127us/step - loss: 0.3609 - acc: 0.8664 - val_loss: 0.5617 - val_acc: 0.7902
Epoch 128/500
569/569 [==============================] 

Epoch 181/500
569/569 [==============================] - 0s 148us/step - loss: 0.3272 - acc: 0.8858 - val_loss: 0.6521 - val_acc: 0.7762
Epoch 182/500
569/569 [==============================] - 0s 121us/step - loss: 0.3378 - acc: 0.8664 - val_loss: 0.6367 - val_acc: 0.7762
Epoch 183/500
569/569 [==============================] - 0s 142us/step - loss: 0.3488 - acc: 0.8612 - val_loss: 0.6278 - val_acc: 0.7762
Epoch 184/500
569/569 [==============================] - 0s 143us/step - loss: 0.3459 - acc: 0.8541 - val_loss: 0.6260 - val_acc: 0.7762
Epoch 185/500
569/569 [==============================] - 0s 122us/step - loss: 0.3397 - acc: 0.8629 - val_loss: 0.6238 - val_acc: 0.7762
Epoch 186/500
569/569 [==============================] - 0s 128us/step - loss: 0.3297 - acc: 0.8682 - val_loss: 0.6466 - val_acc: 0.7762
Epoch 187/500
569/569 [==============================] - 0s 125us/step - loss: 0.3329 - acc: 0.8699 - val_loss: 0.6469 - val_acc: 0.7832
Epoch 188/500
569/569 [==================

Epoch 241/500
569/569 [==============================] - 0s 160us/step - loss: 0.3257 - acc: 0.8682 - val_loss: 0.7352 - val_acc: 0.7762
Epoch 242/500
569/569 [==============================] - 0s 168us/step - loss: 0.3093 - acc: 0.8840 - val_loss: 0.7429 - val_acc: 0.7622
Epoch 243/500
569/569 [==============================] - 0s 161us/step - loss: 0.3165 - acc: 0.8752 - val_loss: 0.7589 - val_acc: 0.7622
Epoch 244/500
569/569 [==============================] - 0s 162us/step - loss: 0.3283 - acc: 0.8805 - val_loss: 0.7658 - val_acc: 0.7552
Epoch 245/500
569/569 [==============================] - 0s 162us/step - loss: 0.3164 - acc: 0.8840 - val_loss: 0.7624 - val_acc: 0.7762
Epoch 246/500
569/569 [==============================] - 0s 152us/step - loss: 0.3089 - acc: 0.8875 - val_loss: 0.7882 - val_acc: 0.7762
Epoch 247/500
569/569 [==============================] - 0s 147us/step - loss: 0.3334 - acc: 0.8752 - val_loss: 0.7688 - val_acc: 0.7622
Epoch 248/500
569/569 [==================

Epoch 301/500
569/569 [==============================] - 0s 157us/step - loss: 0.3072 - acc: 0.8787 - val_loss: 0.9461 - val_acc: 0.7552
Epoch 302/500
569/569 [==============================] - 0s 151us/step - loss: 0.3035 - acc: 0.8787 - val_loss: 0.8750 - val_acc: 0.7622
Epoch 303/500
569/569 [==============================] - 0s 172us/step - loss: 0.3164 - acc: 0.8770 - val_loss: 0.8627 - val_acc: 0.7622
Epoch 304/500
569/569 [==============================] - 0s 129us/step - loss: 0.3100 - acc: 0.8805 - val_loss: 0.8894 - val_acc: 0.7483
Epoch 305/500
569/569 [==============================] - 0s 181us/step - loss: 0.2895 - acc: 0.8928 - val_loss: 0.9144 - val_acc: 0.7622
Epoch 306/500
569/569 [==============================] - 0s 139us/step - loss: 0.3029 - acc: 0.8787 - val_loss: 0.9355 - val_acc: 0.7552
Epoch 307/500
569/569 [==============================] - 0s 162us/step - loss: 0.3063 - acc: 0.8735 - val_loss: 0.9542 - val_acc: 0.7552
Epoch 308/500
569/569 [==================

Epoch 361/500
569/569 [==============================] - 0s 165us/step - loss: 0.2877 - acc: 0.8805 - val_loss: 1.0566 - val_acc: 0.7552
Epoch 362/500
569/569 [==============================] - 0s 148us/step - loss: 0.3037 - acc: 0.8682 - val_loss: 1.0554 - val_acc: 0.7552
Epoch 363/500
569/569 [==============================] - 0s 213us/step - loss: 0.3074 - acc: 0.8822 - val_loss: 1.0341 - val_acc: 0.7552
Epoch 364/500
569/569 [==============================] - 0s 207us/step - loss: 0.3037 - acc: 0.8699 - val_loss: 1.0321 - val_acc: 0.7692
Epoch 365/500
569/569 [==============================] - 0s 164us/step - loss: 0.3241 - acc: 0.8647 - val_loss: 0.9877 - val_acc: 0.7552
Epoch 366/500
569/569 [==============================] - 0s 185us/step - loss: 0.3057 - acc: 0.8717 - val_loss: 0.9179 - val_acc: 0.7832
Epoch 367/500
569/569 [==============================] - 0s 164us/step - loss: 0.3099 - acc: 0.8682 - val_loss: 0.9332 - val_acc: 0.7622
Epoch 368/500
569/569 [==================

Epoch 421/500
569/569 [==============================] - 0s 156us/step - loss: 0.2916 - acc: 0.8858 - val_loss: 1.0253 - val_acc: 0.7692
Epoch 422/500
569/569 [==============================] - 0s 209us/step - loss: 0.2893 - acc: 0.8858 - val_loss: 0.9989 - val_acc: 0.7902
Epoch 423/500
569/569 [==============================] - 0s 138us/step - loss: 0.3017 - acc: 0.8875 - val_loss: 1.0016 - val_acc: 0.7832
Epoch 424/500
569/569 [==============================] - 0s 211us/step - loss: 0.2970 - acc: 0.8805 - val_loss: 0.9971 - val_acc: 0.7832
Epoch 425/500
569/569 [==============================] - 0s 218us/step - loss: 0.2988 - acc: 0.8805 - val_loss: 1.0196 - val_acc: 0.7832
Epoch 426/500
569/569 [==============================] - 0s 151us/step - loss: 0.3066 - acc: 0.8735 - val_loss: 1.0380 - val_acc: 0.7832
Epoch 427/500
569/569 [==============================] - 0s 208us/step - loss: 0.3038 - acc: 0.8682 - val_loss: 1.0202 - val_acc: 0.7762
Epoch 428/500
569/569 [==================

Epoch 481/500
569/569 [==============================] - 0s 183us/step - loss: 0.2879 - acc: 0.8699 - val_loss: 1.0155 - val_acc: 0.7902
Epoch 482/500
569/569 [==============================] - 0s 157us/step - loss: 0.2993 - acc: 0.8840 - val_loss: 1.0040 - val_acc: 0.7902
Epoch 483/500
569/569 [==============================] - 0s 149us/step - loss: 0.3060 - acc: 0.8735 - val_loss: 1.0183 - val_acc: 0.7832
Epoch 484/500
569/569 [==============================] - 0s 168us/step - loss: 0.3145 - acc: 0.8717 - val_loss: 1.0290 - val_acc: 0.7762
Epoch 485/500
569/569 [==============================] - 0s 178us/step - loss: 0.2808 - acc: 0.8875 - val_loss: 0.9790 - val_acc: 0.7832
Epoch 486/500
569/569 [==============================] - 0s 158us/step - loss: 0.2933 - acc: 0.8735 - val_loss: 0.9912 - val_acc: 0.7762
Epoch 487/500
569/569 [==============================] - 0s 163us/step - loss: 0.2832 - acc: 0.8770 - val_loss: 0.9906 - val_acc: 0.7762
Epoch 488/500
569/569 [==================

In [180]:
#del model

In [181]:
y_pred = model.predict(X_test)
#y_pred_train = cl.predict(X_train)
y_pred = (y_pred > 0.5)
#y_pred_train = (y_pred_train > 0.5)


In [182]:
# CM
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
#cm2 = confusion_matrix(y_train, y_pred_train)
prec_test = (cm[0,0] + cm[1,1]) / sum(sum(cm))
#prec_train = (cm2[0,0] + cm2[1,1]) / sum(sum(cm2))
print(prec_test)

0.8268156424581006


In [183]:
y_pred_train = model.predict(X_train)
y_pred_train = (y_pred_train > 0.5)

cm_train = confusion_matrix(y_train, y_pred_train)
#cm2 = confusion_matrix(y_train, y_pred_train)
prec_train = (cm_train[0,0] + cm_train[1,1]) / sum(sum(cm_train))
#prec_train = (cm2[0,0] + cm2[1,1]) / sum(sum(cm2))
print(prec_train)

0.8735955056179775


In [218]:
test.head()


,Sex,Age,Fare,IsChild,SP,Single,SmallF,MedF,LargeF,Title_0,...,T_STONO,T_STONO2,T_STONOQ,T_SWPP,T_WC,T_WEP,T_X,Pc_1,Pc_2,Pc_3
891,1,34.5,7.8292,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
892,0,47.0,7.0000,0,1,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
893,1,62.0,9.6875,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
894,1,27.0,8.6625,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
895,0,22.0,12.2875,0,2,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1


In [249]:
# NN
X = test.values
# X.shape
y = model.predict(X)
print(X.shape)

(418, 65)


In [221]:
y[y>0.5] = 1
y[y<0.5] = 0
y = y.astype(int)

In [187]:
d_test = pd.read_csv('test.csv')
d_test['Survived'] = y
d_test[['PassengerId', 'Survived']].to_csv('results.csv', index=False)
#d_test.info()

In [188]:
from sklearn.ensemble import VotingClassifier
votingC = VotingClassifier(estimators=[('lr', lr), ('rf', clf), ('svc', svm)], voting='soft', n_jobs=4)
votingC = votingC.fit(X_train, y_train)
y_pred_vc = votingC.predict(X_test)

from sklearn.metrics import confusion_matrix
cmvc = confusion_matrix(y_test, y_pred_vc)
precvc = (cmlr[0,0] + cmlr[1,1]) / sum(sum(cmlr))
print(precvc)

y_Vc = votingC.predict(X)

0.8212290502793296


/home/krigare/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/krigare/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [250]:
y_xgb = xgb.predict(X)

/home/krigare/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [191]:
d_test = pd.read_csv('test.csv')
d_test['Survived'] = y
d_test[['PassengerId', 'Survived']].to_csv('resultsLr.csv', index=False)
#d_test.info()

In [251]:
# XGB
d_test = pd.read_csv('test.csv')
d_test['Survived'] = y_xgb
d_test[['PassengerId', 'Survived']].to_csv('resultsXGB.csv', index=False)

In [192]:
d_test = pd.read_csv('test.csv')
d_test['Survived'] = y_rf
d_test[['PassengerId', 'Survived']].to_csv('resultsRf.csv', index=False)

In [193]:
d_test = pd.read_csv('test.csv')
d_test['Survived'] = y_Vc
d_test[['PassengerId', 'Survived']].to_csv('resultsCv.csv', index=False)